In [42]:
EXAMPLE_INPUT = 'RENE=MO10:00-12:00,TU10:00-12:00,TH01:00-03:00,SA14:00-18:00,SU20:00-21:00'

In [21]:
EXAMPLE_INPUT = 'ASTRID=MO10:00-12:00,TH12:00-14:00,SU20:00-21:00'

In [43]:
import re
from datetime import time, datetime, date

In [44]:
re.findall("(\w{2})(\d{2}:\d{2})-(\d{2}:\d{2})", EXAMPLE_INPUT)

[('MO', '10:00', '12:00'),
 ('TU', '10:00', '12:00'),
 ('TH', '01:00', '03:00'),
 ('SA', '14:00', '18:00'),
 ('SU', '20:00', '21:00')]

In [51]:
from datetime import time
from typing import Tuple
Hour = Tuple[int, int]
class Interval:
    def __init__(self, inital_time : Hour, final_time : Hour) -> None:
        if(final_time[0] == 0 and final_time[1] ==0):
            final_time[0] = 24
        assert(final_time > inital_time)
        self.initial_time = inital_time
        self.final_time = final_time
        
    def eliminate_minutes_from_hour(self, hour : Hour) -> float:
        return hour[0] + hour[1]/60

    def in_interval(self, start_time: Hour, end_time : Hour):
        
        
        return (
            (self.eliminate_minutes_from_hour(start_time) - self.eliminate_minutes_from_hour(self.initial_time)) >=0) and (
            self.eliminate_minutes_from_hour(end_time) - self.eliminate_minutes_from_hour(self.final_time) <=0
        )






In [210]:
## Default intervals
first_interval = Interval((0, 1),  (9, 0))
second_interval = Interval((9, 1), (18, 0))
third_interval = Interval((18, 1), (24, 0))

intervals = [first_interval, second_interval, third_interval]

In [211]:
from enum import Enum

class DefaultTagEnum(Enum):
    WeekDay = 0
    Weekend = 1

In [212]:
payment_configuration= {
    DefaultTagEnum.WeekDay : {
        first_interval : 25,
        second_interval : 15,
        third_interval : 20
    }, 
    
    DefaultTagEnum.Weekend: {
        first_interval : 30,
        second_interval : 20,
        third_interval : 25
    }
}

In [213]:
RE_PATTERN ="(\w{2})(\d{2}:\d{2})-(\d{2}:\d{2})"
def parse_times(current_input : str) -> Tuple[str, Hour, Hour]:
    for data in re.findall(RE_PATTERN, EXAMPLE_INPUT):
        day, start, end = data
        
        start_hour, start_minute = start.split(':')
        start_hour, start_minute = int(start_hour), int(start_minute)
    
        end_hour, end_minute = end.split(':')
        end_hour, end_minute = int(end_hour), int(end_minute)
        
        yield (day, (start_hour, start_minute), (end_hour, end_minute))
        
    

In [214]:
def get_total_earned_on_day(input_str: str):
    total_earned = 0
    for day, start_time, end_time in parse_times(input_str):
        period_worked_hours = end_time[0] - start_time[0] + (end_time[1] - start_time[1])/60

        matched_intervals = [interval for interval in intervals if interval.in_interval(start_time, end_time)]

        assert(len(matched_intervals) == 1)

        matched_interval = matched_intervals[0]

        if(re.match('MO|TU|WE|TH|FR', day)):
            total_earned += payment_configuration[DefaultTagEnum.WeekDay][matched_interval] * period_worked_hours

        elif (re.match('SA|SU', day)):
            total_earned += payment_configuration[DefaultTagEnum.Weekend][matched_interval] * period_worked_hours

        else:
            raise ValueError("Day does not exist")
            
    return total_earned
    
get_total_earned_on_day(EXAMPLE_INPUT)

215.0

total_earned = 0
for data in re.findall("(\w{2})(\d{2}:\d{2})-(\d{2}:\d{2})", EXAMPLE_INPUT):
    day, start, end = data
    start_hour, start_minute = start.split(':')
    start_hour, start_minute = int(start_hour), int(start_minute)
    
    
    end_hour, end_minute = end.split(':')
    end_hour, end_minute = int(end_hour), int(end_minute)
    if(end_hour == 0):
        end_hour = 24
    
    period_worked_hours = end_hour - start_hour
    
    
    matched_interval = [i for i in  intervals if i.in_interval( 
            (start_hour, start_minute), (end_hour, end_minute)
    )]
    
    assert(len(matched_interval) == 1)
    
    matched_interval = matched_interval[0]
    
    
    if(re.match('MO|TU|WE|TH|FR', day)):
        total_earned += payment_configuration[DefaultTagEnum.WeekDay][matched_interval] * period_worked_hours
        
    elif (re.match('SA|SU', day)):
        total_earned += payment_configuration[DefaultTagEnum.Weekend][matched_interval] * period_worked_hours
        
    else:
        print("_______ERRROR______")
    
    print(data)
    print(third_interval.in_interval((start_hour, start_minute), (end_hour, end_minute)))

total_earned

time(0, 0) == time(0,0)

In [261]:
from isort import Config
from DataStructures.DefaultTagEnum import DefaultTagEnum
from DataStructures.Interval import Interval
from random import random

class ConfigManager:
    default_tag_price : dict[DefaultTagEnum, dict[Interval, int]] =None
        # DefaultTagEnum.WeekDay : [25,15,20],
        # DefaultTagEnum.Weekend : [30, 20, 25]
    # }

    intervals = []


    custom_tag_price : dict[str, dict[Interval, int]] = {}

    @staticmethod
    def register_tag_price(tag : str,prices_by_interval: dict[Interval, int]):
        ConfigManager.custom_tag_price[tag] = prices_by_interval

    @staticmethod
    def register_interval(interval : Interval):
        if(ConfigManager.default_tag_price is not None): raise ValueError("You can't register any more intervals as the configuration already began")
        if(interval in ConfigManager.intervals): raise ValueError("Interval is already registered")

        overrided_intervals = [i for i in ConfigManager.intervals if i.in_interval(interval.initial_time, interval.initial_time) or i.in_interval(interval.final_time, interval.final_time)]
        
        if(len(overrided_intervals) > 0):
            print(overrided_intervals)
            raise ValueError("Interval is overriding one already registered")
        ConfigManager.intervals.append(interval)
        

    @staticmethod
    def register_configuration(config : dict[DefaultTagEnum, dict[Interval, int]]):
        for tag in config:
            configured_intervals = [*config[tag].keys()]
            
        assert(set(configured_intervals) == set(ConfigManager.intervals))
                




In [262]:
ConfigManager.intervals = []
print(ConfigManager.intervals)

[]


In [263]:
ConfigManager.register_interval(first_interval)

In [264]:
ConfigManager.register_interval(second_interval)

In [265]:
ConfigManager.register_interval(third_interval)

In [266]:
ConfigManager.intervals

In [268]:
ConfigManager.register_configuration(
    {DefaultTagEnum.WeekDay : {
        first_interval:2,
        second_interval: 3,
        third_interval: 4
    }
     })